In [1]:
# import relevant libaries 
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import glob 
from vincenty import vincenty
import datetime

import geopandas
import json
# conda install h3-py -c conda-forge
from h3 import h3
from shapely.geometry import shape
from descartes import PolygonPatch

import folium

from folium import plugins
from folium.plugins import HeatMapWithTime
from folium.plugins import MarkerCluster
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle


from h3 import h3
from shapely.geometry import shape
from descartes import PolygonPatch

# import timeit to measure execution times
import timeit

#imports of folium for map visualization
import folium
from folium import plugins
from folium.plugins import HeatMap

#import Matplotlib Ticker to style axes of figures
import matplotlib.ticker as ticker

import warnings
warnings.filterwarnings(action='ignore')

# Where do the trips start most often?

In [2]:
#creates hexagons for the city of Bonn 
#returns geodataframe with h3_hex_ids and geometries of the hexagons
def hexForBonn(resolution=9):
    city_bounding_box = geopandas.read_file('Data/BonnGeoJSON.geojson')
    city_bounding_box_json_string = city_bounding_box.to_json()
    city_bounding_box_json = json.loads(city_bounding_box_json_string)
    city_bounding_box_poly = city_bounding_box_json["features"][0]
    H3_HEXAGON_RESOLUTION = resolution
    polygon_obj = city_bounding_box_poly["geometry"]
    hexagons = h3.polyfill(geo_json = polygon_obj, res = H3_HEXAGON_RESOLUTION, geo_json_conformant = True)
    # Convert H3 Indexes to Geometry Objects
    # Create geo data frame
    df_geos = geopandas.GeoDataFrame(list(hexagons), columns=['h3_hex_id'], crs="EPSG:4326")#contains h3_hex_ids and geometries
    # transform h3 to geo_boundary 
    df_geos['geometry'] = df_geos[(df_geos['h3_hex_id'].notna())].apply(
        lambda row: shape({"type": "Polygon",
                           "coordinates": [h3.h3_to_geo_boundary(h3_address=row["h3_hex_id"], geo_json=True)],
                           "properties": ""
                          }), axis=1)
    df_geos.to_file("data/hex.geojson", driver="GeoJSON") # save hex with specified hex_resolution to use it in choropleth 
    return df_geos

def trips_per_stations_in_hex(resolution=9): 
    trips = pd.read_csv("Data/bonn_trips.csv",index_col=0) # read the trip data 
    trips["count"] = 1 
    trips = trips[(trips['p_spot_start']) & (trips['p_spot_end'] != 0)] # filter on official stations 
    
    # group on stations and sum trips  
    trips_per_station = trips.groupby(["latitude_start", "longitude_start", "p_name_start"]).sum()[["count"]]
    trips_per_station.reset_index(inplace=True)
    trips_per_station.rename(columns={"latitude_start":"latitude", 
                                      "longitude_start":"longitude", 
                                      "p_name_start":"station_name",
                                      "count":"trips"}, inplace=True)
    
    # creating geodataframe of trips_per_station    
    geometry = [Point(xy) for xy in zip(trips_per_station['longitude'], trips_per_station['latitude'])]
    gdf_trips_per_station = geopandas.GeoDataFrame(trips_per_station, crs='epsg:4326', geometry=geometry)
    # join with hexagons 
    df_geos = hexForBonn(resolution=resolution) #create hexagons for bonn 
    gdf_trips_per_station = geopandas.sjoin(df_geos, gdf_trips_per_station, how='inner', op='contains')
    gdf_trips_per_station.reset_index(inplace=True)
    
    # group by hex_id and sum up all trips per hexagons
    gdf_trips_per_station = gdf_trips_per_station.groupby("h3_hex_id").sum()
    gdf_trips_per_station.reset_index(inplace=True
                                     )
    # returns 2 geodataframe (hexagons for bonn / trips_per_hexagons)
    return df_geos, gdf_trips_per_station

# returns choropleth 
def choropleth_trip_amounts_per_hex(hex_resolution=8):
    df_geos, trips = trips_per_stations_in_hex(hex_resolution) # calls functions above 
    m = folium.Map(location = [50.7323,7.1847], zoom_start=11.2,min_zoom=10)
    folium.Choropleth(geo_data="data/hex.geojson",
                      data=trips,
                      columns=["h3_hex_id","trips"],
                      key_on="properties.h3_hex_id",
                      fill_color='Reds',
                      fill_opacity=0.7,
                      line_opacity=0.5,
                      line_weight=1,
                      line_color='black',
                      legend_name='Number of bookings per hexagon').add_to(m)    
    

    return m 

In [3]:
# with the parameter "hex_resolution" you can vary the size of the hexagons 
choropleth_trip_amounts_per_hex(hex_resolution=8)

# Availabilty of bikes per station to any time 

In [4]:
def checkValues(day,month,hour):
    month31 = [1,3,5,7,8,10,12]
    if month == 2: 
        if (day < 0) | (day>28): 
            print ("No valid date")
    if month in month31: 
        if (day < 0) | (day>31): 
            print ("No valid date")
    else: 
        if (day < 0) | (day>30): 
            print ("No valid date")
        
    if (month < 0) | (month >12): 
        print ("No valid date")
    
    if (hour<0) | (hour>24):
        print ("No valid time")
    
    if month == 1: 
        if day < 20: 
            print ("No data for this time available")

def formatValues(day,month,hour): 
    if day < 10: 
        d = '0' + str(day)
    else: 
        d = str(day)
    if month < 10: 
        m = '0' + str(month)
    else: 
        m = str(month)

    if hour < 10: 
        h = '0' + str(hour)
    else: 
        h = str(hour)
    return d,m,h

In [5]:
# returns a choropleth showing the number of available bikes per hexagon 
# parameter: resolution = vary the size of hexagons 
#            day, month, hour: specify the datetime 

def choropleth_available_bikes_per_hex(day=1,month=3,hour=12, resolution = 9): 
    # create hexagons for bonn with specified resolution 
    df_geos = hexForBonn(resolution=resolution)
    
    df = pd.read_csv("Data/compressedRawData.csv",index_col=0) # read rawdata 
    
    # delete the "non-stations"
    df = df[df['p_spot']]
    df.loc[:,'datetime'] = pd.to_datetime(df.loc[:, 'datetime'])
    
    # store station information in one df for later merging 
    station_geos = pd.DataFrame(data=df["p_name"].unique(), columns=["station_name"])
    station_geos = pd.merge(station_geos, df, 
                            left_on="station_name", 
                            right_on="p_name",how="left")[["station_name","p_lat","p_lng"]]

    #drop duplicates that are obtained by the merge 
    station_geos.drop_duplicates(subset=["station_name"],keep="first",inplace=True,ignore_index=True)
    station_geos.rename({"p_lat":"latitude","p_lng":"longitude"},axis=1,inplace =True)
    
    # transform to geodataframe 
    geometry= [Point(xy) for xy in zip(station_geos['longitude'], station_geos['latitude'])]
    station_geos = geopandas.GeoDataFrame(station_geos, crs='epsg:4326', geometry=geometry)
    station_geos.reset_index(inplace=True,drop=True)
    
    station_geos = geopandas.sjoin(df_geos,station_geos,how='inner', op='contains') # merge with the hexagons 
    station_geos.drop(["index_right"], inplace=True,axis=1)
    station_geos.reset_index(inplace=True,drop=True) # contains stations, geo information and the corresponding hexagons 
    
    stationlist = df['p_name'].unique() # list of official bike stations 

    # Create dataframe with hourly timestamps
    df_availableBikes = pd.DataFrame(np.arange('2019-01-01', '2019-12-31', dtype='datetime64[h]'), columns = ['time'])
    df_availableBikes['time'] = df_availableBikes['time'].dt.strftime('%Y-%m-%d-%H')

    for station in stationlist: 
        df_availableBikes[station] = np.NaN
        
    df_availableBikes = df_availableBikes.set_index('time')
    
    # dataframe that contains a row for every hour in the year
    for index, row in df.iterrows():
        # fill values for each hour 
        timestamp = pd.to_datetime(row.get(key = 'datetime')).ceil('H').strftime('%Y-%m-%d-%H')
        df_availableBikes.at[timestamp, row.get(key = 'p_name')] = pd.to_numeric(row.get(key = 'p_bikes'))
    
    # fill NaN values with 0 
    df_availableBikes.fillna(method='ffill', inplace=True)
    df_availableBikes.fillna(value='0.0', inplace=True)

    # transform values to floats to be able to map them in a choropleth
    for i in df_availableBikes.columns: 
        df_availableBikes[i] = pd.to_numeric(df_availableBikes[i])
        
    checkValues(day,month, hour) #check if day, month and hour are correct inputs
    
    d, m, h = formatValues(day,month,hour) # format the values to string to be able to access the data 
        
    date_filter = '2019-' + m + "-" + d + "-" + h # build a datefilter 
    
    # this df below contains the results for the number of available bikes to a specified datetime
    data_by_date = pd.DataFrame(df_availableBikes.loc[date_filter]) 
    data_by_date.reset_index(inplace=True)
    data_by_date = data_by_date.rename(columns={'index': 'p_name',date_filter:"available_bikes"})

    # merge the result with the station_geos to be able to map them in a choropleth 
    data_by_date = pd.merge(data_by_date, station_geos, left_on="p_name", right_on="station_name")
    data_by_date.drop("p_name", inplace=True, axis=1)
    
    # group over h3_hex_id to map the sum of available bikes per hexagon 
    data_by_date =  data_by_date.groupby("h3_hex_id").sum()
    data_by_date.reset_index(drop=False, inplace=True)
    
    m = folium.Map(location = [50.7323,7.1847], zoom_start=11, min_zoom=10.5, max_zoom=16)
    folium.Choropleth(geo_data="data/hex.geojson",   # display all hexagons 
                        data=data_by_date,           # df with available bikes per hex
                        columns=["h3_hex_id","available_bikes"],
                        key_on="properties.h3_hex_id", # key of the data and geodata is the hex id 
                        fill_color='Reds',
                        fill_opacity=0.7,
                        line_opacity=0.5,
                        line_weight=1,
                        line_color='black',
                        legend_name='Number of available bikes per hexagon',
                        bins=7
                    ).add_to(m) 
    return m 

In [6]:
# displays the number of available bikes on 01.04.2019 at 4PM 
# hexagon of hexagons is 6 
choropleth_available_bikes_per_hex(day=1,month=4,hour=16,resolution=6)

In [7]:
# displays the number of available bikes on 01.04.2019 at 4PM 
# hexagon of hexagons is 7
choropleth_available_bikes_per_hex(1,4,16,7)

In [8]:
# displays the number of available bikes on 01.04.2019 at 4PM 
# hexagon of hexagons is 8 
choropleth_available_bikes_per_hex(1,4,16,8)

In [9]:
# displays the number of available bikes on 01.04.2019 at 4PM 
# hexagon of hexagons is 9
choropleth_available_bikes_per_hex(1,4,16,9)